In [1]:
# This path append is just for testing from within the examples folder, no need for the user to add this
import sys
sys.path.append("/home/hweiner/Internship/CoMET_rams_devel/")

Define the CONFIG String

In [2]:
CONFIG_str = """
# ===========================================Welcome To CoMET===========================================
# This is where most all functionality of CoMET can be specified. You can change any of the supplied 
# variables OR you can use a default suite of paramateres setup for different model and observation
# input types. 

# TODO: Add CONFIG setup to CoMET-UDAF specification.


# SETUP VARIABLES: These determine basic CoMET functionality
verbose: True # Whether to use verbose output
parallel_processing: True # [bool] Whether or not to use parallel processing for certain tasks
max_cores: 32 # Number of cores to use if parallel_processing==True; Enter None for unlimited

# Structered in this form:
# Observation Type:
#   path_to_data
#   additional_observation_parameters
#
#   tracker:
#       tracker_params
#
#       analysis:
#           analysis_variables

# WRF
wrf:
    path_to_data: "/D3/data/hweiner/Model_Test_Data/WRF/*"
    
    is_idealized: False
#     #min_frame_index: 10 # 0-based indexing, inclusive
#     #max_frame_index: 70 # 0-based indexing, inclusive

    feature_tracking_var: "wa" #DBZ, TB, WA, PR, Or other WRF Standard variable name (Case-Sensitive)
    segmentation_var: "wa"

    tobac:
        feature_id:
            threshold: [3, 5, 10]
            target: "maximum"
            position_threshold: "weighted_diff"
            sigma_threshold: 0.5
            n_min_threshold: 4
            #detect_subset: {1:(29, 47)}

        linking:
            method_linking: "predict"
            adaptive_stop: 0.2
            adaptive_step: 0.95
            order: 1
            subnetwork_size: 10
            memory: 0
            v_max: 20
            time_cell_min: 120

        segmentation_2d:
            height: 6 # km
            method: "watershed"
            target: 'maximum'
            threshold: 3

        segmentation_3d:
            method: "watershed"
            target: 'maximum'
            threshold: 3
            seed_3D_flag: 'box'
    
        analysis: # Optional
            merge_split: { variable: "WA" }
            area: { height: 6 }
            volume: {}
            max_intensity: { variable : "WA", cell_footprint_height: 6}
            var_max_height: {threshold: 3, cell_footprint_height : 6, variable: "WA"}
            velocity: {}
            perimeter: {}
            cell_growth: {variable: "volume"}
            irregularity: {irregularity_metrics: ['sphericity']}

"""

Run CoMET

In [3]:
import CoCoMET

CONFIG = CoCoMET.CoCoMET_load(CONFIG_string=CONFIG_str)
print(CONFIG, "\n")

output = CoCoMET.CoCoMET_start(CONFIG=CONFIG)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

===============Welcome To CoCoMET===============

A toolkit of the Advanced Study of Cloud and Environment iNTerations (ASCENT) program.

This project was supported by the U.S. Department of Energy (DOE) Early Career Research Program, Atmospheric System Research (ASR) program, and the Office of Workforce Development for Teachers and Scientists (WDTS) under the Science Undergraduate Laboratory Internships Program (SULI).

If you are using this software for a publication, please cite: ####


=====WRF Setup Found in CONFIG=====
{'verbose': True, 'para

=====Performing tobac Segmentation to US=====: 100%|██████████| 48/48 [04:15<00:00,  5.32s/it]


=====Starting WRF tobac Analysis Calculations=====


=====Calculating Cell Sphericities=====: 100%|██████████| 17745/17745 [00:00<00:00, 197031.67it/s]


=====Converting WRF tobac Output to CoCoMET-US=====
=====WRF tobac Tracking Complete=====
=====CoCoMET Performance Diagonistics=====
$ Total Process Time: 2410.47 Seconds
$ Allocated Resources: Cores = 32


Save Data

In [5]:
import json
import pickle
from copy import deepcopy
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

# Make a copy of the outcopy as to not corrupt it when saving
outcopy = deepcopy(output)

# Define the path where to save the files
savepath = "/D3/data/hweiner/test_saving_data"

for dataset in outcopy.keys():

    for tracker in outcopy[dataset].keys():
        print(f"Now saving {dataset} data with {tracker} tracker")

        # Save the tracks and segmentation information
        tracks_copy = deepcopy(outcopy[dataset][tracker]["US_tracks"]) 

        # the csv cannot hold datetime objects, so transform into seconds
        tracks_copy["lifetime"] = tracks_copy["lifetime"].dt.total_seconds()
        tracks_copy.to_csv(f"{savepath}/{dataset.upper()}_Tracks_{tracker}.csv", index = False)

        print("\t Saving tracks information")

        # Save the segmentation if there is any
        if "US_segmentation_3d" in outcopy[dataset][tracker] and outcopy[dataset][tracker]["US_segmentation_3d"] is not None:

            segmentation_3d = deepcopy(outcopy[dataset][tracker]["US_segmentation_3d"])

            with open(f"{savepath}/{dataset.upper()}_segmentation_3d_{tracker}.pickle", "wb") as file:
                pickle.dump(segmentation_3d, file)
            
            print("\t Saving the 3D segmentation information")


        if "US_segmentation_2d" in outcopy[dataset][tracker] and outcopy[dataset][tracker]["US_segmentation_2d"] is not None:

            segmentation_2d = deepcopy(outcopy[dataset][tracker]["US_segmentation_2d"])

            with open(f"{savepath}/{dataset.upper()}_segmentation_2d_{tracker}.pickle", "wb") as file:
                pickle.dump(segmentation_2d, file)

            print("\t Saving the 2D segmentation information")


        # Save the analysis data in a dictionary
        if "analysis" in outcopy[dataset][tracker] and len(outcopy[dataset][tracker]["analysis"]) != 0:

            print(f"\t Now saving analysis variables:")

            for analysis_variable in outcopy[dataset][tracker]["analysis"].keys():

                if analysis_variable in ["var_max_height"]:

                    outcopy[dataset][tracker]["analysis"][analysis_variable]["max_height"].to_csv(f"{savepath}/{dataset}_{tracker}_{analysis_variable}.csv", index=False)
                    print(f"\t \t {analysis_variable}")

                elif analysis_variable in ["irregularity"]:
                    
                    # go through all of the irregularity measurements in analysis
                    for irregularity_key in outcopy[dataset][tracker]["analysis"][analysis_variable].keys()[3:]: # skip frame, feature id, cell id
                        
                        outcopy[dataset][tracker]["analysis"][analysis_variable][irregularity_key].to_csv(f"{savepath}/{dataset}_{tracker}_{irregularity_key}.csv", index=False)
                        print(f"\t \t {irregularity_key}")

                elif analysis_variable == "merge_split":
                    
                    outcopy[dataset][tracker]["analysis"][analysis_variable][0].to_csv(f"{savepath}/{dataset}_{tracker}_mergers.csv", index=False)
                    outcopy[dataset][tracker]["analysis"][analysis_variable][1].to_csv(f"{savepath}/{dataset}_{tracker}_splitters.csv", index=False)
                    print("\t \t merge/split information")

                else:
                    
                    outcopy[dataset][tracker]["analysis"][analysis_variable].to_csv(f"{savepath}/{dataset}_{tracker}_{analysis_variable}.csv", index=False)
                    print(f"\t \t {analysis_variable}")

Now saving wrf data with tobac tracker
	 Saving tracks information
	 Saving the 3D segmentation information
	 Saving the 2D segmentation information
	 Now saving analysis variables:
	 	 area
	 	 perimeter
	 	 volume
	 	 merge/split information
	 	 max_intensity
	 	 var_max_height
	 	 velocity
	 	 cell_growth
	 	 sphericity


Open Data

In [1]:
import pandas as pd
import pickle
import os

path_to_files = "/D3/data/hweiner/test_saving_data"

# Open the tracks and segmentation data
WRF_tracks = pd.read_csv(f"{path_to_files}/WRF_Tracks_tobac.csv")

with open(f'{path_to_files}/WRF_segmentation_3d_tobac.pickle', 'rb') as data:
    WRF_segmentation_3d = pickle.load(data)

In [2]:
# Open the analysis data by joining all of the dataframes together
    # Does not work for merging / splitting - will have to open merging / splitting separately
wrf_analysis_df = pd.DataFrame({})
for file in os.listdir(path_to_files):
    filename = os.fsdecode(file)
    if ("wrf_tobac" in filename and ".csv" in filename
        and "mergers" not in filename
        and "splitters" not in filename):
        if len(wrf_analysis_df) < 3:
            wrf_analysis_df = pd.read_csv(f"{path_to_files}/{filename}")
        else:
            f = pd.read_csv(f"{path_to_files}/{filename}")
            cols = set(f.keys()) - set(['frame', 'feature_id', 'cell_id'])
            for c in cols:
                wrf_analysis_df = wrf_analysis_df.join(f[c])

wrf_mergers = pd.read_csv(f"{path_to_files}/wrf_tobac_mergers.csv")
wrf_splitters = pd.read_csv(f"{path_to_files}/wrf_tobac_splitters.csv")

In [3]:
wrf_analysis_df

,frame,feature_id,cell_id,area,perimeter,volume,max_intensity,max_height,velocity unit vector,speed,cell_growth,sphericity
0,0,0,0,NaN,0,0.204075,NaN,NaN,[-3.27724021e-04 -7.07106743e-01 -7.07106743e-01],2.357023,0.000830,NaN
1,0,1,1,NaN,0,0.449292,NaN,NaN,[-7.50926663e-04 -7.57900960e-01 -6.52369198e-01],2.554791,0.000581,NaN
2,0,2,2,NaN,0,1.249166,NaN,NaN,[-7.15690739e-04 -7.95019489e-01 -6.06583465e-01],3.088653,-0.000930,NaN
3,0,3,3,NaN,0,0.906497,NaN,NaN,[-0.00112497 -0.26422492 -0.96446043],2.549559,0.001454,NaN
4,0,4,4,NaN,0,0.401012,NaN,NaN,[-5.13016400e-04 -8.70257370e-01 -4.92597044e-01],2.960326,0.001767,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
17740,47,17740,4962,NaN,0,4.199359,NaN,NaN,NaN,NaN,NaN,NaN
17741,47,17741,3437,NaN,0,2.374813,NaN,NaN,NaN,NaN,NaN,NaN
17742,47,17742,4810,NaN,0,3.433893,NaN,NaN,NaN,NaN,NaN,NaN
17743,47,17743,4743,NaN,0,1.374616,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
WRF_tracks

,frame,time,lifetime,lifetime_percent,feature_id,cell_id,south_north,west_east,up_down,latitude,longitude,projection_x,projection_y,altitude,geometry
0,0,2013-06-19 16:00:00,0.0,0.0,0,0,190.000000,160.000000,12.877010,29.203678,-95.541931,80250.000000,95250.000000,0.875882,POINT (-95.54193115234375 29.203678131103516)
1,0,2013-06-19 16:00:00,0.0,0.0,1,1,200.000000,97.000000,12.310065,29.247555,-95.871887,48750.000000,100250.000000,0.824871,POINT (-95.87188720703125 29.247554779052734)
2,0,2013-06-19 16:00:00,0.0,0.0,2,2,133.481139,159.679255,17.775160,28.945612,-95.542196,80089.627293,66990.569636,1.382769,POINT (-95.54219607677221 28.945612241411492)
3,0,2013-06-19 16:00:00,0.0,0.0,3,3,416.754157,350.000000,16.985108,30.239502,-94.543274,175250.000000,208627.078641,1.292208,POINT (-94.54327392578125 30.23950184673192)
4,0,2013-06-19 16:00:00,0.0,0.0,4,4,175.000000,145.000000,14.056053,29.134834,-95.619965,72750.000000,87750.000000,0.986942,POINT (-95.61996459960938 29.13483428955078)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17740,47,2013-06-19 19:55:00,600.0,1.0,17740,4962,487.118660,213.784600,31.600267,30.561361,-95.264178,107142.299853,243809.329941,3.679982,POINT (-95.26417841525239 30.561361233283634)
17741,47,2013-06-19 19:55:00,3300.0,1.0,17741,3437,254.307267,149.000000,28.603853,29.497065,-95.601236,74750.000000,127403.633479,3.044183,POINT (-95.60123618368402 29.49706543102198)
17742,47,2013-06-19 19:55:00,900.0,1.0,17742,4810,423.000000,353.412426,28.325984,30.267956,-94.525105,176956.212826,211750.000000,2.989661,POINT (-94.52510529669944 30.267955543028798)
17743,47,2013-06-19 19:55:00,900.0,1.0,17743,4743,464.162226,352.000000,30.394872,30.455966,-94.531672,176250.000000,232331.112765,3.413111,POINT (-94.53167229536345 30.45596579469043)


In [17]:
WRF_segmentation_3d

<xarray.Dataset> Size: 9GB
Dimensions:                  (time: 48, up_down: 94, south_north: 498,
                              west_east: 498)
Coordinates:
  * time                     (time) datetime64[ns] 384B 2013-06-19T16:00:00 ....
  * up_down                  (up_down) int64 752B 0 1 2 3 4 5 ... 89 90 91 92 93
  * south_north              (south_north) int64 4kB 0 1 2 3 ... 494 495 496 497
  * west_east                (west_east) int64 4kB 0 1 2 3 4 ... 494 495 496 497
    altitude                 (up_down) float32 376B 11.69 48.94 ... 2.07e+04
    latitude                 (south_north, west_east) float32 992kB 28.33 ......
    longitude                (south_north, west_east) float32 992kB -96.37 .....
    projection_x_coordinate  (west_east) float64 4kB 250.0 750.0 ... 2.488e+05
    projection_y_coordinate  (south_north) float64 4kB 250.0 750.0 ... 2.488e+05
Data variables:
    Feature_Segmentation     (time, up_down, south_north, west_east) int32 4GB ...
    Cell_Segmentation        (time, up_down, south_north, west_east) int32 4GB ...